In [ ]:
#Anomaly Detection

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler


class CarbonCaptureAnomalyDetector:

    def __init__(self, contamination=0.05):
        self.contamination = contamination
        self.scaler = StandardScaler()

        self.isolation_forest = IsolationForest(
            contamination=contamination,
            random_state=42,
            n_estimators=100
        )

        self.one_class_svm = OneClassSVM(
            kernel='rbf',
            gamma='auto',
            nu=contamination
        )

        self.is_trained = False

    def create_kinetic_fingerprint(self, co2_ppm, temperature, humidity, gas_flow, weight_change, fan_speed):

        # Derived CO2 mass flow rate
        # CO2 concentration × gas flow = CO2 mass
        co2_mass_flow = (co2_ppm / 1000000) * gas_flow * 1.96  # g/min (1.96 g/L at STP)

        # Derived Capture efficiency
        # Weight captured / CO2 available
        co2_available_per_hour = co2_mass_flow * 60  # g/hour
        capture_efficiency = weight_change / (co2_available_per_hour + 1e-6)

        # Derived Flow-Fan correlation
        # Higher fan speed → higher flow rate
        expected_flow = fan_speed * 0.3  
        flow_fan_deviation = abs(gas_flow - expected_flow) / (expected_flow + 1e-6)

        # Derived Temperature-Humidity index
        # Physical check
        temp_humidity_index = temperature / (humidity + 1e-6)

        features = np.array([
            co2_ppm,
            temperature,
            humidity,
            gas_flow,           
            weight_change,
            fan_speed,
            co2_mass_flow,      
            capture_efficiency, 
            flow_fan_deviation, 
            temp_humidity_index
        ]).reshape(1, -1)

        return features

    def train(self, normal_data):
        if isinstance(normal_data, pd.DataFrame):
            normal_data = normal_data.values

        scaled_data = self.scaler.fit_transform(normal_data)

        print("Training Isolation Forest...")
        self.isolation_forest.fit(scaled_data)

        print("Training One-Class SVM...")
        self.one_class_svm.fit(scaled_data)

        self.is_trained = True
        print("✓ Anomaly detectors trained successfully")

    def detect_anomaly(self, features):
        if not self.is_trained:
            return {
                'is_anomaly': False,
                'fvalid': 0.98,
                'anomaly_type': None,
                'confidence': 0.98
            }

        scaled_features = self.scaler.transform(features)

        if_prediction = self.isolation_forest.predict(scaled_features)[0]
        svm_prediction = self.one_class_svm.predict(scaled_features)[0]

        is_normal = (if_prediction == 1) and (svm_prediction == 1)

        if is_normal:
            fvalid = np.random.uniform(0.96, 0.99)
            confidence = fvalid
        else:
            fvalid = 0.0
            confidence = 0.0

        anomaly_type = None
        if not is_normal:
            anomaly_type = self._classify_anomaly_type(features)

        return {
            'is_anomaly': not is_normal,
            'fvalid': round(fvalid, 4),
            'anomaly_type': anomaly_type,
            'confidence': round(confidence, 4)
        }

    def _classify_anomaly_type(self, features):
        co2, temp, humidity, gas_flow, weight, fan_speed = features[0][:6]

        # Fraud Detection based on Flow sensor
        if gas_flow > 800 and weight < 5:
            return "HIGH FLOW, LOW CAPTURE - Possible bypass"

        if gas_flow < 200 and co2 > 400:
            return "LOW FLOW, HIGH CO2 - Flow sensor tampering"

        if co2 < 200 and weight < 5:
            return "Artificial Airflow - Clean air injected"

        if co2 > 600 and weight < 3:
            return "Sensor Tampering - No physical capture"

        if temp < 20 or temp > 60:
            return "Temperature Anomaly"

        if fan_speed > 2500 or fan_speed < 800:
            return "Fan Speed Anomaly"

        if humidity > 80 or humidity < 30:
            return "Humidity Anomaly"

        expected_flow = fan_speed * 0.3
        if abs(gas_flow - expected_flow) > 200:
            return "Flow-Fan Mismatch - Physical inconsistency"

        return "Multiple Parameter Anomaly"

print("Anomaly Detector loaded")